# word2vec without eager

参考：

http://www.jeyzhang.com/tensorflow-learning-notes-3.html

https://blog.csdn.net/linxuheng/article/details/70162356

### word embedding

将源数据映射到另外一个空间，
单词嵌入，就是把X所属空间的单词映射为到Y空间的多维向量，那么该多维向量相当于嵌入到Y所属空间中。

给出一个文档，文档就是一个单词序列比如 “A B A C B F G”, 希望对文档中每个不同的单词都得到一个对应的向量(往往是低维向量)表示。
比如，对于这样的“A B A C B F G”的一个序列，也许我们最后能得到：A对应的向量为[0.1 0.6 -0.5]，B对应的向量为[-0.2 0.9 0.7] （此处的数值只用于示意）
之所以希望把每个单词变成一个向量，目的还是为了方便计算，比如“求单词A的同义词”，就可以通过“求与单词A在cos距离下最相似的向量”来做到。

比较简单的方法：
基于BOW的one-hot；
缺点：没有相邻单词的信息，向量可能会非常的长

#### 共现矩阵Cocurrence matrix：

向量空间模型(Vector space models, VSMs)将词语表示为一个连续的词向量，并且语义接近的词语对应的词向量在空间上也是接近的。VSMs在NLP神经语言程序学 (Neuro-Linguistic Programming) 中拥有很长的历史，但是所有的方法在某种程度上都是基于一种分布式假说，该假说的思想是如果两个词的上下文(context)相同，那么这两个词所表达的语义也是一样的；换言之，两个词的语义是否相同或相似，取决于两个词的上下文内容，上下文相同表示两个词是可以等价替换的

分布式假说中一个非常重要的思想是，我们认为某个词的意思跟它临近的单词是紧密相关的。这是我们可以设定一个窗口（大小一般是5~10），如下窗口大小是2，那么在这个窗口内，与rests 共同出现的单词就有life、he、in、peace。然后我们就利用这种共现关系来生成词向量。
<img src="https://img-blog.csdn.net/20170904134137027" style="width:450px;height:100px">
例如，现在我们的语料库包括下面三份文档资料：
I like deep learning. 
I like NLP. 
I enjoy flying.
作为示例，我们设定的窗口大小为1，也就是只看某个单词周围紧邻着的那个单词。此时，将得到一个对称矩阵——共现矩阵。因为在我们的语料库中，I 和 like做为邻居同时出现在窗口中的次数是2，所以下表中I 和like相交的位置其值就是2。这样我们也实现了将word变成向量的设想，在共现矩阵每一行（或每一列）都是对应单词的一个向量表示。

<img src="https://img-blog.csdn.net/20170904134757679" style="width:400px;height:300px">

为了缩小向量，会采用SVD或PCA等降维方法。但是SVD操作计算量巨大(此处不赘述，以后有空补充)

而深度学习流行之后，Tomas Mikolov 提出了Word2vec。

Wrod2vec是一个典型的预测模型，用于高效地学习Word Embedding。实现的模型有两种：连续词袋模型(CBOW)和Skip-Gram模型。算法上这两个模型是相似的，只不过CBOW是从输入的上下文信息来预测目标词(例如利用 [the cat sits on the] 来预测 [mat] )；而skip-gram模型则是相反的，从目标词来预测上下文信息。一般而言，这种方式上的区别使得CBOW模型更适合应用在小规模的数据集上，能够对很多的分布式信息进行平滑处理；而Skip-Gram模型则比较适合用于大规模的数据集上。

## Skip-Gram模型 

skip-gram模型是从目标词来预测上下文信息。因此相似的词将会有相似的数值表达（相似的词向量）。

我们先定义如下的符号：

V - 词汇表的大小（语料库当中唯一词的数量）

P - 投影层或向量层

D - 向量空间的维度

b - 单个Batch的大小

我们假设的模型是简单的逻辑回归（Softmax）模型。因此高级的框架如下所示。下面给出了两个图，左边的是概念框架，右边的是实现框架。虽然框架不同，但是它们没有对功能做任何改变。
<img src="http://i1.wp.com/www.thushv.com/wp-content/uploads/2016/12/skip_gram.png" style="width:650px;height:350px">

为了方便说明，我们假设处理的句子为 The dog barked at the mailman 。第一个模型即正在对数据 (input:'dog',output:['the','barked','at','the','mailman']) 进行训练的模型，它们共享Softmax层的权重(weights)和偏差(biases)。换句化说，概念模型训练对同一个输入同时训练多个输出。然而这实际上很难实现。因此我们通常把元组 (input:'dog',output:['the','barked','at','the','mailman']) 拆分成为单个的二元组，如 (input:'dog', output:'the'),(input:'dog', output:'barked'),...,(input:'dog', output:'mailman') ，也就是右边图片的做法。

并且，为了提高模型训练速度，我们还采用负采样函数NCE。
### 噪声对比估计损失函数(noise-contrastive estimation (NCE))
常用的逻辑回归函数，sotfmax，定义如下：
<img src="./picture/softmax.png" style="width:450px;height:45px">

但是显然，softmax需要遍历字典上所有的单词，因此每一步的训练代价都是O(VocabularySize)。为了减少训练代价，有两种方式：hierarchical softmax 和 sample-based softmax。其中，negative sampling （归属于sample-based softmax）实质为NCE（noise-contrastive estimation）

简单来说，目标函数的意义是显然的，即尽可能的 [分配(assign)] 高概率给真实的目标词，而低概率给其他[噪声词]，这种技术称为负采样(Negative Sampling)。同时，该目标函数具有很好的数学意义：即在条件限制(训练时间)的情况下尽可能的逼近原有的Softmax函数，这样做无疑能够大大提升模型训练的速度。（具体见论文NCE.pdf，不想看。）

但是，谨记，NCE只能在训练模型的时候使用，真实预测的时候还是要用softmax！

tensorflow中的对应函数为：

tf.nn.nce_loss(weights, biases, labels, inputs, num_sampled, num_classes, 
    num_true=1, sampled_values=None, remove_accidental_hits=False, 
    partition_strategy='mod', name='nce_loss')

## Skip-Gram with tensorflow

### 1. 导入包

TensorBoard 的一个内置的可视化工具 Embedding Projector, 是个交互式的可视化，可用来分析诸如 embeddings 的高维数据。 

embedding projector 将从你的 checkpoint 文件中读取 embeddings。 

默认情况下，embedding projector 会用 PCA 主成分分析方法将高维数据投影到 3D 空间, 还有一种投影方法是 T-SNE。

In [1]:
""" starter code for word2vec skip-gram model with NCE loss
CS 20: "TensorFlow for Deep Learning Research"
cs20.stanford.edu
Chip Huyen (chiphuyen@cs.stanford.edu)
Lecture 04
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector #版本更新，embedding投影工具
import tensorflow as tf

import utils
import word2vec_utils   #这两个文件在example文件夹下，别忘了加

Instructions for updating:
Use the retry module or similar alternatives.


### 2.设置超参数

In [2]:
# Model hyperparameters
VOCAB_SIZE = 50000          #词典的大小
BATCH_SIZE = 128            #每批次处理的大小
EMBED_SIZE = 128            # dimension of the word embedding vectors
SKIP_WINDOW = 1             # the context window
NUM_SAMPLED = 64            # number of negative examples to sample
LEARNING_RATE = 1.0         #优化器的学习率
NUM_TRAIN_STEPS = 100000     #训练次数
VISUAL_FLD = 'visualization'
SKIP_STEP = 5000

### 3. 下载数据

关于python的注释：

1、yield使用   
1）函数中使用yield，可以使函数变成生成器。一个函数如果是生成一个数组，就必须把数据存储在内存中，如果使用生成器，则在调用的时候才生成数据，可以节省内存。  
2）生成器方法调用时，不会立即执行。需要调用next()或者使用for循环来执行。使用for循环不需要自己捕获StopIteration异常。使用next()方法，当生产器方法执行结束会抛出StopIteration异常（只要不是使用yield返回数据，都会抛出StopIteration异常）。   
3）yield不仅可以返回值，也可以接收值。  
4）调用生成器send方法传递数据时，必须先调用next(c)或者c.send(None)方法，执行到yield语句，等待接收数据。否则会报错。  
2、yield from的使用   
1）为了让生成器（带yield函数），能简易的在其他函数中直接调用，就产生了yield from。   
2）假设，htest为生成器，itest通过yield from 直接调用htest。这样itest也变成了一个生成器。创建itest实例不断的去获取数据，当生成器执行结束时，会抛出StopIteration异常。那这个异常是htest抛出的，还是itest抛出的。通过捕获异常，会发现其实是itest抛出异常，htest并不会抛出StopIteration异常。   
3)yield from 也可以返回值，通过变量接收。变量接收的值，即htest使用return返回的值。

In [3]:
# Parameters for downloading data
DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016
NUM_VISUALIZE = 3000        # number of tokens to visualize

def gen():
    yield from word2vec_utils.batch_gen(DOWNLOAD_URL, EXPECTED_BYTES, VOCAB_SIZE, 
                                        BATCH_SIZE, SKIP_WINDOW, VISUAL_FLD)

### 4. 建立模型并训练

In [4]:
def word2vec(dataset):
    """ Build the graph for word2vec model and train it """
    # Step 1: get input, output from the dataset
    with tf.name_scope('data'):
        iterator = dataset.make_initializable_iterator()
        center_words, target_words = iterator.get_next()

    """ Step 2 + 3: define weights and embedding lookup.
    In word2vec, it's actually the weights that we care about 
    """
    with tf.name_scope('embed'):
        #定义一下词嵌入矩阵(Embedding Matrix)，并随机初始化
        embed_matrix = tf.get_variable('embed_matrix', 
                                        shape=[VOCAB_SIZE, EMBED_SIZE],
                                        initializer=tf.random_uniform_initializer())
        embed = tf.nn.embedding_lookup(embed_matrix, center_words, name='embedding')

    # Step 4: construct variables for NCE loss and define loss function
    with tf.name_scope('loss'):
        #需要定义词库中每个词的权值和偏置参数(称为输出层权值参数)，用于NCE
        nce_weight = tf.get_variable('nce_weight', shape=[VOCAB_SIZE, EMBED_SIZE],
                        initializer=tf.truncated_normal_initializer(stddev=1.0 / (EMBED_SIZE ** 0.5)))
        nce_bias = tf.get_variable('nce_bias', initializer=tf.zeros([VOCAB_SIZE]))

        # define loss function to be NCE loss function
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weight, 
                                            biases=nce_bias, 
                                            labels=target_words, 
                                            inputs=embed, 
                                            num_sampled=NUM_SAMPLED, 
                                            num_classes=VOCAB_SIZE), name='loss')

    # Step 5: define optimizer
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
    
    utils.safe_mkdir('checkpoints')

    with tf.Session() as sess:
        sess.run(iterator.initializer)
        sess.run(tf.global_variables_initializer())

        total_loss = 0.0 # we use this to calculate late average loss in the last SKIP_STEP steps
        writer = tf.summary.FileWriter('graphs/word2vec_simple', sess.graph)

        for index in range(NUM_TRAIN_STEPS):
            try:
                loss_batch, _ = sess.run([loss, optimizer])
                total_loss += loss_batch
                if (index + 1) % SKIP_STEP == 0:
                    print('Average loss at step {}: {:5.1f}'.format(index, total_loss / SKIP_STEP))
                    total_loss = 0.0
            except tf.errors.OutOfRangeError:
                sess.run(iterator.initializer)
        writer.close()

### 5.主函数

In [6]:
def main():
    dataset = tf.data.Dataset.from_generator(gen, 
                                (tf.int32, tf.int32), 
                                (tf.TensorShape([BATCH_SIZE]), tf.TensorShape([BATCH_SIZE, 1])))
    word2vec(dataset)

if __name__ == '__main__':
    main()

data/text8.zip already exists
Average loss at step 4999:  65.3
Average loss at step 9999:  18.3
Average loss at step 14999:   9.8
Average loss at step 19999:   6.6
Average loss at step 24999:   5.7
Average loss at step 29999:   5.2
Average loss at step 34999:   5.0
Average loss at step 39999:   4.9
Average loss at step 44999:   4.8
Average loss at step 49999:   4.8
Average loss at step 54999:   4.8
Average loss at step 59999:   4.7
Average loss at step 64999:   4.6
Average loss at step 69999:   4.7
Average loss at step 74999:   4.6
Average loss at step 79999:   4.6
Average loss at step 84999:   4.7
Average loss at step 89999:   4.7
Average loss at step 94999:   4.6
Average loss at step 99999:   4.6
